In [1]:
import albumentations as A
import cv2
import json
import numpy as np
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.data.image_augmentation import get_bbox_from_json, plot_img_with_bbox

In [2]:
transform = A.Compose([
    A.RandomCrop(width=450, height=450),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
], bbox_params=A.BboxParams(format='coco'))

In [15]:
folder_path = "../data/raw/nano/Rot"
trans_img_bbox = []
COUNT = 10

if os.path.isdir(folder_path):
    for obj in os.listdir(folder_path):
        obj_path = os.path.join(folder_path, obj)
        # Search for JSON-File in folder
        if os.path.isfile(obj_path) and obj.lower().endswith(".json"):
            json_file_path = obj_path
            # Check JPG-Files in folder
            for obj in os.listdir(folder_path):
                obj_path = os.path.join(folder_path, obj)
                if os.path.isfile(obj_path) and obj.lower().endswith(".jpg"):
                    #Get image and bbox
                    img = cv2.imread(obj_path)
                    bbox = get_bbox_from_json(json_file_path, obj)
                    # Transform image with bbox multiple times
                    for i in range(0, COUNT):
                        transformed = transform(image=img, bboxes=bbox)
                        trans_img_bbox.append(transformed)

In [16]:
#transformed_image = transformed['image']
#transformed_bboxes = transformed['bboxes']

for i in range(0,8):
    plot_img_with_bbox(transformed_image=trans_img_bbox[i]['image'], transformed_bboxes=trans_img_bbox[i]['bboxes'])